In [1]:
def convert_rgb_hsl(r,g,b): #function to convert rgb values to hsl values
    R = r / 255
    G = g / 255
    B = b / 255
    minimum = min([R,G,B])
    maximum = max([R,G,B])
    l = ((minimum+maximum)/2)*100
    if l>50:
        s = ((maximum-minimum)/(2.0-maximum-minimum))*100
    else:
        s = ((maximum-minimum)/(maximum+minimum))*100

    if maximum == R:
        h = ((G-B)/(maximum-minimum))*60
    elif maximum == G:
        h = (2.0 + ((B-R)/(maximum-minimum)))*60
    else:
        h = (4.0 + ((R-G)/(maximum-minimum)))*60
    return (h,s,l)

In [2]:
def getTrainingLabels(file_list): #for hand-labeling input data
    labels = file_list
    counter = 0
    for file in file_list:
        im = Image.open(file)
        im.show()
        labels[counter] = input("Label: ")
        counter += 1
    return labels
        

## Below is a list of data labels for train/test data

In [3]:
y = ['okay', 'good', 'okay', 'okay', 'good', 'good', 'good', 'okay', 'good', 'good', 'good', 'okay', 'good', 'okay', 'bad', 'bad', 'bad', 'good', 'good', 'okay', 'bad', 'good', 'okay', 'okay', 'okay', 'good', 'okay', 'okay', 'okay', 'okay', 'good', 'good', 'good', 'bad', 'good', 'good', 'okay', 'okay', 'okay', 'good', 'bad', 'bad', 'bad', 'good', 'good', 'okay', 'bad', 'bad', 'good', 'okay', 'good', 'good', 'okay', 'okay', 'okay', 'good', 'okay', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'okay', 'good', 'good', 'good', 'okay', 'good', 'okay', 'good', 'okay', 'okay', 'good', 'good', 'good', 'good', 'good', 'good', 'okay', 'okay', 'okay', 'good', 'okay', 'okay', 'okay', 'good', 'good', 'okay', 'okay', 'good', 'okay', 'good', 'good', 'good', 'good', 'okay', 'good', 'good', 'good', 'okay', 'okay', 'okay', 'good', 'good', 'good', 'good', 'good', 'good', 'okay', 'good', 'okay', 'good', 'good', 'good', 'good', 'okay', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'okay', 'good', 'good', 'okay', 'okay', 'good', 'good', 'bad', 'okay', 'okay', 'good', 'bad', 'okay', 'okay', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'okay', 'good', 'okay', 'okay', 'bad', 'okay', 'okay', 'bad', 'bad', 'okay', 'good', 'good', 'bad', 'good', 'okay', 'good', 'good', 'okay', 'good', 'okay', 'okay', 'good', 'good', 'good', 'good', 'good', 'good', 'okay', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'okay', 'good', 'good', 'good', 'good', 'good', 'okay', 'okay', 'okay', 'good', 'okay', 'okay', 'okay', 'good', 'good', 'okay', 'good', 'good', 'good', 'good']

In [30]:
def Read_Image(filename): 
    # gets features for a given image, which is number of shades of blue and red and the number of pixels of each
    im = Image.open(filename) #opens given image file
    pixel_values = list(im.getdata()) #loads pixel data from image in RGBA format
    dict_colors = {} #creates a dictionary of pixel values
    for pixel in pixel_values:
        if pixel not in dict_colors: #if a pixel value is not in the list add it
            dict_colors[pixel] = 1
        else: #increase count by one if it appears again
            dict_colors[pixel] += 1
    blue_red = [0,0] #stores number of blues (0) and reds (1)
    
    #want the number of blue and red pixels because we don't want too much white background
    
    numpixels = [0,0] #stores number of blue pixels (0) and red pixels (1)
    
    #Since we want to see color gradients and not a lot of pure blue and red, store number of pixels of pure colors
    if (0,0,255,255) in dict_colors.keys(): 
        blue = dict_colors[(0,0,255,255)]
    else:
        blue = 0
        
    if (255,0,0,255) in dict_colors.keys():
        red = dict_colors[(255,0,0,255)]
    else:
        red = 0
        
    solids = [blue, red] #solid blue and red count
    for key in dict_colors.keys():
        #the if statement makes sure the color is not black, white, or gray
        #and checks if the key has more than 20 appearences to account for non-visible color changes
        if  dict_colors[key] > 20 and key != (255,255,255,255) and key != (0,0,0,255)and key[0]!=key[2]:
            (h,s,l) = convert_rgb_hsl(key[0], key[1], key[2]) #converts to hsl to help determine color
            if l> 20: # also makes sure value is not gray
                if h>(170) and h<(300):#color is blue
                    blue_red[0] +=1
                    numpixels[0] += dict_colors[key]
                else:#color is red
                    blue_red[1] +=1
                    numpixels[1] += dict_colors[key]
    return blue_red, numpixels, solids
        

In [13]:
def getFeatures(file_list): #gets features for all given images
    feature_list =[[]]*len(file_list)
    counter = 0
    for file in file_list:
        [blue_red, num_pixels, solids] = Read_Image(file)
        blue_red.append(num_pixels[0])
        blue_red.append(num_pixels[1])
        blue_red.append(solids[0])
        blue_red.append(solids[1])
        feature_list[counter]=blue_red
        counter+=1
    return feature_list

In [14]:
import os
from PIL import Image
file_list = os.listdir(os.curdir)
file_list = file_list[2:218] #file_list includes all png files
x = getFeatures(file_list)

# Use KNN to Classify Images

In [27]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [28]:
from sklearn import metrics
p = knn.predict(x_test)
a=metrics.accuracy_score(p, y_test)
r=metrics.classification_report(p,y_test)
print(a)
print(r)

0.863636363636
             precision    recall  f1-score   support

        bad       1.00      1.00      1.00         7
       good       0.83      0.90      0.86        21
       okay       0.86      0.75      0.80        16

avg / total       0.87      0.86      0.86        44



|      |precision  |  recall | f1-score  | support|
|------|-----------|---------|-----------|--------|
|bad   |   1.00    |  1.00   |  1.00     |   7    |
|------|-----------|---------|-----------|------- |
|good  |   0.83    |  0.90   |  0.86     |   21   |
|------|-----------|---------|-----------|--------|
|okay  |   0.86    |  0.75   |  0.80     |   16   |
|------|-----------|---------|-----------|--------|
|total |   0.87    |  0.86   |  0.86     |   44   |

## Accuracy score = 0.863636363636

In [44]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
def getUsableFiles(file_list, y_train, x_train): # predicts classification of files and returns good and okay ones
    x_test = getFeatures(file_list)
    knn = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')
    knn.fit(x_train, y_train)
    predictions = knn.predict(x_test)
    good_okay = []
    for i in range(len(predictions)):
        if (predictions[i] == 'good') or (predictions[i] == 'okay'):
            good_okay.append(file_list[i])
    return good_okay

In [47]:
y_train = ['okay', 'good', 'okay', 'okay', 'good', 'good', 'good', 'okay', 'good', 'good', 'good', 'okay', 'good', 'okay', 'bad', 'bad', 'bad', 'good', 'good', 'okay', 'bad', 'good', 'okay', 'okay', 'okay', 'good', 'okay', 'okay', 'okay', 'okay', 'good', 'good', 'good', 'bad', 'good', 'good', 'okay', 'okay', 'okay', 'good', 'bad', 'bad', 'bad', 'good', 'good', 'okay', 'bad', 'bad', 'good', 'okay', 'good', 'good', 'okay', 'okay', 'okay', 'good', 'okay', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'okay', 'good', 'good', 'good', 'okay', 'good', 'okay', 'good', 'okay', 'okay', 'good', 'good', 'good', 'good', 'good', 'good', 'okay', 'okay', 'okay', 'good', 'okay', 'okay', 'okay', 'good', 'good', 'okay', 'okay', 'good', 'okay', 'good', 'good', 'good', 'good', 'okay', 'good', 'good', 'good', 'okay', 'okay', 'okay', 'good', 'good', 'good', 'good', 'good', 'good', 'okay', 'good', 'okay', 'good', 'good', 'good', 'good', 'okay', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'okay', 'good', 'good', 'okay', 'okay', 'good', 'good', 'bad', 'okay', 'okay', 'good', 'bad', 'okay', 'okay', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'okay', 'good', 'okay', 'okay', 'bad', 'okay', 'okay', 'bad', 'bad', 'okay', 'good', 'good', 'bad', 'good', 'okay', 'good', 'good', 'okay', 'good', 'okay', 'okay', 'good', 'good', 'good', 'good', 'good', 'good', 'okay', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'okay', 'good', 'good', 'good', 'good', 'good', 'okay', 'okay', 'okay', 'good', 'okay', 'okay', 'okay', 'good', 'good', 'okay', 'good', 'good', 'good', 'good']
file_list = os.listdir(os.curdir)
file_list = file_list[2:218] #file_list includes all png files
usable = getUsableFiles(file_list, y_train, x_train)
#to view the images:
#for image in usable:
    #im = Image.open(image)
    #im.show()